# Loading the model 

In [1]:
from tensorflow.keras.models import load_model
model = load_model(r'dummy.model')

# Initialize camera and face identifier

In [2]:
import tensorflow as tf
import cv2
import numpy

#starting the video stream
cap = cv2.VideoCapture(0) 

#using the XML file for haarcascade classifier
clf = cv2.CascadeClassifier(r"haarcascade_frontalface_default.xml")

# Detecting the mask

In [3]:
#using the loop to watch the stream in real time.
while True:
    (success, frame) = cap.read()  #reading the frame from the stream 
    new_img = cv2.resize(frame, (frame.shape[1] // 1, frame.shape[0] // 1)) #resizing the frame to speed up the process of detection
    face = clf.detectMultiScale(new_img) #detecting faces from the frame(ROI)
    for x,y,w,h in face:
        try:
            #getting the coordinates for the face detected
            face_img = new_img[y:x+h, x:x+w]
            #resizing the  face detected to fit into the model in the shape(224,224)
            resized = cv2.resize(face_img,(224,224))
            #converting the detected image into an array
            image_array = tf.keras.preprocessing.image.img_to_array(resized) 
            #expanding the dimensions to fit in the model
            image_array = tf.expand_dims(image_array,0) 
            #making predictions on the ROI
            predictions = model.predict(image_array) 
            #getting the results
            score = tf.nn.softmax(predictions[0])
            label = numpy.argmax(score)
        except Exception as e:
            print('bad frame')
            
        if label == 0:
            cv2.rectangle(new_img,(x,y),(x+w,y+h),(0,255,0),2)
            cv2.putText(new_img,"mask",(x,y),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,0), 2)
        elif label == 1:
            cv2.rectangle(new_img,(x,y),(x+w,y+h),(0,0,255),2)
            cv2.putText(new_img,'no_mask',(x,y),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255), 2)
        else:
            None
    #displaying the window after predicting the outcome
    cv2.imshow('face_window', new_img)
    #waitkey to terminate the loop
    key = cv2.waitKey(10) 
    if key == ord('q'):
        break
#release the stream 
cap.release()
cv2.destroyAllWindows()